<a href="https://colab.research.google.com/github/dagpadraget/rakdnngit/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
cd /

/


In [36]:
cd content

/content


In [0]:
rm -fr rakdnngit

In [38]:
!git clone https://github.com/dagpadraget/rakdnngit

Cloning into 'rakdnngit'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 66 (delta 29), reused 52 (delta 15), pack-reused 0
Unpacking objects: 100% (66/66), done.


In [0]:
!mkdir tmp

In [40]:
!pwd

/content


In [41]:
cd rakdnngit

/content/rakdnngit


In [42]:
!ls -la

total 7272
drwxr-xr-x 5 root root    4096 Jan  5 13:55 .
drwxr-xr-x 1 root root    4096 Jan  5 13:56 ..
-rw-r--r-- 1 root root 7410460 Jan  5 13:55 dagmymodel.model
drwxr-xr-x 8 root root    4096 Jan  5 13:55 .git
drwxr-xr-x 2 root root    4096 Jan  5 13:55 localdata
-rw-r--r-- 1 root root     513 Jan  5 13:55 main.py
-rw-r--r-- 1 root root     178 Jan  5 13:55 requirement.txt
-rw-r--r-- 1 root root     245 Jan  5 13:55 run_local
drwxr-xr-x 2 root root    4096 Jan  5 13:55 trainer


In [43]:
!pip3 install -r requirement.txt

In [47]:
!python3 main.py

Using TensorFlow backend.
number of sequences= 141 

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
2020-01-05 13:59:21.699279: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-01-05 13:59:21.721423: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-01-05 13:59:21.722004: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla P4 major: 6 minor: 1 memoryClockRate(GHz): 1.1135
pciBusID: 0000:00:04.0
2020-01-05 13:59:21.722291: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-01-05 13:59:21.723764: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-01-05 1

In [45]:
pwd

'/content/rakdnngit'

In [0]:
more localdata/hbshort.txt